# Example of similarity score calculation of 2CO on Pt(553) configurations

### Import the necessary functions

In [1]:
import pandas as pd
import time # timing the execution of the similarity calculation
import similarity as sim
print("successfully loaded packages")

successfully loaded packages


### Load the configurations
There are 2 pairs of conifgurations (total of 4 configurations) subjected to the similarity calculation.\
The configurations in conf_pair1 (conf1 and conf2) are highly similar.\
The configurations in conf_pair2 (conf3 and conf4) are highly dis-similar.
conf1             |  conf2
:-------------------------:|:-------------------------:
![](conf_img/conf1.png)  |  ![](conf_img/conf2.png)

conf3             |  conf4
:-------------------------:|:-------------------------:
![](conf_img/conf3.png)  |  ![](conf_img/conf4.png)

In [2]:
conf1_path = '2COconf1.CONTCAR'
conf2_path = '2COconf2.CONTCAR'
conf3_path = '2COconf3.CONTCAR'
conf4_path = '2COconf4.CONTCAR'

conf_pair1 = [conf1_path,conf2_path]
conf_pair2 = [conf3_path,conf4_path]

### Do the similarity calculations

In [3]:
start = time.time()
conf_pair1_score = sim.compare_eigval_diff(conf_pair1,start_atom_ele='Pt')
conf_pair2_score = sim.compare_eigval_diff(conf_pair2,start_atom_ele='Pt')
end = time.time()
t_execution = end - start

print(f'conf_pair1 score is {conf_pair1_score}')
print(f'conf_pair2 score is {conf_pair2_score}')
print(f'execution time is {t_execution} s')

conf_pair1 score is -0.00010181489051319659
conf_pair2 score is -0.24061769247055054
execution time is 1.3150207996368408 s
